# LING 281 Project

## Data Analysis

In [3]:
import pandas as pd
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [4]:
df = pd.read_csv('usages_unpacked_lowered_trimmed.csv', dtype=str)
df.head(20)

,session,birth_year,home_location,sex,caretaker,visit_frequency,ibu_curr_home_lang,ibu_past_home_lang,ibu_work_lang,ibu_neighbor_lang,...,di_sd_dengan_teman_sekelas_lang,di_sma_dengan_guru_lang,di_sma_dengan_teman_sekelas_lang,di_kendaraan_umum_lang,membaca_novel_lang,membaca_koran_lang,membaca_majalah_lang,mendengar_musik_lang,mendengar_radio_lang,menonton_tv_lang
0,imqyyl,1991,jawa barat - kota depok,laki,orang tua,NaN,NaN,NaN,NaN,NaN,...,indonesia,indonesia,indonesia,indonesia,indonesia,inggris,inggris,inggris,indonesia,indonesia
1,xhmqvm,1993,banten - kota tangerang,laki,orang tua,lebih dari 24,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,indonesia,NaN,NaN,inggris,NaN,NaN
2,oqhpqe,1995,jawa timur - kota surabaya,perempuan,orang tua,12-24,jawa,jawa,indonesia,indonesia,...,"indonesia, jawa",indonesia,indonesia,indonesia,"indonesia, inggris","indonesia, inggris",indonesia,"indonesia, inggris","indonesia, inggris","indonesia, inggris"
3,ssxypn,1993,dki jakarta,laki,keluarga,lebih dari 24,"indonesia, minang",minang,indonesia,indonesia,...,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia
4,iggaxc,1965,dki jakarta,perempuan,orang tua,1-2,"indonesia, sunda",sunda,indonesia,indonesia,...,indonesia,indonesia,indonesia,indonesia,"indonesia, inggris","indonesia, inggris","indonesia, inggris","indonesia, inggris","indonesia, inggris","indonesia, inggris"
5,lmlldp,1986,dki jakarta,perempuan,orang tua,12-24,"jawa, betawi",jawa,"jawa, indonesia",indonesia,...,betawi,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia
6,mhtlse,1989,dki jakarta - kota administrasi jakarta selatan,laki,NaN,3-5,indonesia,jawa,indonesia,indonesia,...,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia
7,udifdh,1971,jawa timur - kabupaten tulungagung,laki,keluarga,NaN,NaN,NaN,NaN,NaN,...,jawa,indonesia,jawa,indonesia,"indonesia, inggris","indonesia, inggris","indonesia, inggris","indonesia, inggris",indonesia,"indonesia, inggris"
8,juvfub,1989,jawa barat - kota bogor,perempuan,orang tua,12-24,indonesia,"sunda, jawa",indonesia,"indonesia, sunda",...,indonesia,indonesia,indonesia,indonesia,"mampu baca, tapi saya nggak berkomuniklasi den...","indonesia, inggris","indonesia, inggris","indonesia, inggris","indonesia, inggris","indonesia, inggris"
9,xsqxfc,1989,dki jakarta - kota administrasi jakarta timur,laki,orang tua,lebih dari 24,"indonesia, jawa, manado",NaN,"indonesia, jawa","indonesia, jawa",...,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia,"indonesia, inggris","indonesia, jawa, inggris, melayu","indonesia, inggris","indonesia, inggris"


In [16]:
lang_df = df

for header in df:
    if 'lang' not in header:
        lang_df = lang_df.drop(columns=[header])

lang_df

,ibu_curr_home_lang,ibu_past_home_lang,ibu_work_lang,ibu_neighbor_lang,ibu_partner_lang,ibu_family_lang,ibu_first_lang,ibu_other_langs,bapak_curr_home_lang,bapak_past_home_lang,...,di_sd_dengan_teman_sekelas_lang,di_sma_dengan_guru_lang,di_sma_dengan_teman_sekelas_lang,di_kendaraan_umum_lang,membaca_novel_lang,membaca_koran_lang,membaca_majalah_lang,mendengar_musik_lang,mendengar_radio_lang,menonton_tv_lang
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,indonesia,indonesia,indonesia,indonesia,indonesia,inggris,inggris,inggris,indonesia,indonesia
1,NaN,NaN,NaN,NaN,NaN,NaN,betawi,indonesia,NaN,NaN,...,NaN,NaN,NaN,NaN,indonesia,NaN,NaN,inggris,NaN,NaN
2,jawa,jawa,indonesia,indonesia,jawa,jawa,indonesia,jawa,jawa,jawa,...,"indonesia, jawa",indonesia,indonesia,indonesia,"indonesia, inggris","indonesia, inggris",indonesia,"indonesia, inggris","indonesia, inggris","indonesia, inggris"
3,"indonesia, minang",minang,indonesia,indonesia,"indonesia, minang",minang,minang,minang,"indonesia, minang",minang,...,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia,indonesia
4,"indonesia, sunda",sunda,indonesia,indonesia,"indonesia, sunda","indonesia, sunda",sunda,indonesia,"indonesia, sunda","indonesia, sunda",...,indonesia,indonesia,indonesia,indonesia,"indonesia, inggris","indonesia, inggris","indonesia, inggris","indonesia, inggris","indonesia, inggris","indonesia, inggris"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,jawa,jawa,NaN,jawa,jawa,"indonesia, jawa",jawa,indonesia,jawa,jawa,...,jawa,"indonesia, jawa","indonesia, jawa",jawa,"inggris, indonesia, jawa","indonesia, inggris",indonesia,"indonesia, inggris",indonesia,"indonesia, jawa"
656,jawa,jawa,NaN,jawa,jawa,jawa,jawa,indonesia,jawa,jawa,...,"jawa, indonesia",indonesia,"jawa, indonesia",indonesia,"indonesia, inggris",indonesia,indonesia,"indonesia, inggris",indonesia,indonesia
657,jawa,jawa,NaN,jawa,jawa,jawa,jawa,indonesia,jawa,jawa,...,"jawa, indonesia",indonesia,"jawa, indonesia",jawa,"indonesia, inggris","indonesia, inggris","indonesia, inggris","indonesia, inggris",indonesia,"indonesia, inggris"
658,"jawa, indonesia",jawa,"jawa, indonesia",jawa,"jawa, indonesia",indonesia,"jawa, indonesia",jawa,jawa,jawa,...,"jawa, indonesia",indonesia,"jawa, indonesia",indonesia,"indonesia, inggris",indonesia,indonesia,"indonesia, inggris",indonesia,"indonesia, inggris"


In [40]:
import re

def langs_from_entry(entry):
    if pd.isna(entry):
        return set()
    else:
        entry = entry.strip()
        langs = re.split(' *, *', entry)
        return set(langs)

In [41]:
lang_labels = set()

for header in lang_df:
    for entry in lang_df[header]:
        lang_labels |= langs_from_entry(entry)

lang_labels

{'',
 '"alien"',
 '(campur)',
 '(campur) korea',
 'a',
 'abui - mainang',
 'abui - munumawati',
 'abui - takalelang',
 'aceh',
 'adat',
 'alay',
 'alay (minang)',
 'alm',
 'almarhum',
 'alor',
 'ambon',
 'angka',
 'apa',
 'apa kabar',
 'apa nek',
 'api',
 'arab',
 'argamakmur',
 'asing',
 'ass',
 'awak',
 'b. daerah',
 'b. isyarat',
 'b.inggris',
 'bahasa',
 'bahsa inggris',
 'bak-bak',
 'baku',
 'bali',
 'bali alus',
 'bali indonesia',
 'balikpapan',
 'banci',
 'bandar lampung',
 'bangka',
 'banjar',
 'barat',
 'baserah',
 'basma',
 'batak',
 'batak karo',
 'batak toba',
 'batin',
 'bebas',
 'belanda',
 'belu',
 'bengkulu',
 'bengkulu "o"',
 'bengkulu (lembak)',
 'bengkulu (o)',
 'bengkulu lembak',
 'bengkulu selatan',
 'berdoa dalam hati',
 'besemah',
 'betawi',
 'bhs indonesia',
 'bima',
 'bima (ntb)',
 'bkkbn',
 'bkl',
 'blitar',
 'blm punya suami',
 'bpsp pertanian',
 'brazil',
 'bugis',
 'bugis/palakka',
 'buk',
 'buk mau kemana',
 'bukittinggi',
 'bulog',
 'bunak',
 'bungah',
 '